In [ ]:
#import all libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import zipfile
import logging

In [ ]:
def get_links(input_url):
    pass #TODO

In [ ]:
def get_soup(input_url):
    headers = {
        'User-Agent': 'My User Agent 1.0',
        "Accept-Language": "en-US,en;q=0.5"
    }
    response = requests.get(input_url, headers=headers)
    if response.status_code != 200:
        print("Error in getting link")
        print("response code is : ", response.status_code)
    site_soup = BeautifulSoup(response.text, 'html.parser')
    return site_soup

In [ ]:
def get_fa_title(soup):
    pass #TODO

In [ ]:
def get_en_title(soup):
    pass #TODO

In [ ]:
def get_price(soup):
    pass

In [1]:
def get_discount(soup):
    pass

In [ ]:
def get_score(soup):
    pass

In [ ]:
def get_publisher(soup):
    pass

In [ ]:
def get_author(soup):
    pass

In [ ]:
def get_code(soup):
    pass

In [ ]:
def get_isbn(soup):
    pass

In [ ]:
def get_pages(soup):
    pass

In [ ]:
def get_per_date(soup):
    pass

In [ ]:
def get_ad_date(soup):
    pass

In [ ]:
def get_size(soup):
    pass

In [ ]:
def get_material(soup):
    pass

In [ ]:
data_list = []
url = "temp"  #TODO
links = get_links(url)
soups = get_soup(links)

for site_soup in soups:
    try:
        book_fa_title = get_fa_title(site_soup)
        book_en_title = get_en_title(site_soup)
        book_price = get_price(site_soup)
        book_discount = get_discount(site_soup)
        book_score = get_score(site_soup)
        book_publisher = get_publisher(site_soup)
        book_author = get_author(site_soup)
        book_code = get_code(site_soup)
        book_Isbn = get_isbn(site_soup)
        book_pages = get_pages(site_soup)
        book_publication_per_date = get_per_date(site_soup)
        book_publication_ad_date = get_ad_date(site_soup)
        book_size = get_size(site_soup)
        boo_cover_material = get_material(site_soup)

        data = "Temp"  #TODO
        data_list += [data]
    except Exception:
        logging.exception("An error occurred")